In [1]:
# TODO:add bit arrays / hash values to json file

In [23]:
import json
with open('data/wikipedia_20000_frequencies.json', 'r') as f:
    frequencies = json.load(f)

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

In [5]:
import pandas as pd
en_wiki_frequencies = pd.read_csv('data/enwiki-2023-04-13.txt', header=None, sep=" ", index_col=0)

In [6]:
valid_entries = en_wiki_frequencies.index.map(lambda x: bool(wordnet.synsets(str(x))))
en_wiki_frequencies = en_wiki_frequencies.loc[valid_entries]
en_wiki_frequencies = en_wiki_frequencies[pd.notnull(en_wiki_frequencies.index)].to_dict()

In [7]:
en_wiki_frequencies = en_wiki_frequencies[1]

In [8]:
with open('data/lemma_safe_enwiki_frequencies.json', 'r') as f:
    combined_frequencies = json.load(f)

In [24]:
for word in frequencies.keys():
    for adjacent_word in frequencies[word].keys():
        if frequencies[word][adjacent_word] < 5:
            frequencies[word][adjacent_word] = 0
            continue

        total_frequency = 0
        if adjacent_word not in combined_frequencies.keys():
            for entry in en_wiki_frequencies.keys():
                if lemmatizer.lemmatize(str(entry)) == adjacent_word:
                    total_frequency += int(en_wiki_frequencies[entry])
            print('total', adjacent_word, total_frequency)
            combined_frequencies[adjacent_word] = total_frequency
        else:
            total_frequency = combined_frequencies[adjacent_word]
        
        if total_frequency:
            frequencies[word][adjacent_word] /= float(total_frequency)
        else:
            frequencies[word][adjacent_word] = 0.0

In [25]:
import mmh3

def hash_digests(token, bits):
    return [mmh3.hash(token, i) % bits for i in range(3)]

In [26]:
for key in frequencies.keys():
    frequencies[key] = dict(sorted(frequencies[key].items(), key=lambda item: item[1], reverse=True))
for key in frequencies.keys():
    for word in frequencies[key].keys():
        frequencies[key][word] = {
            'tf-idf': frequencies[key][word],
            'bloom_filter': list(hash_digests(word, 32))
        }

In [27]:
# removes newlines in lists for readability
import re
def repl_func(match: re.Match):
    return " ".join(match.group().split())

json_str = json.dumps(frequencies, indent=4)
json_str = re.sub(r"(?<=\[)[^\[\]]+(?=])", repl_func, json_str)

with open('data/wikipedia_20000_tf-idf.json', 'w') as f:
    f.write(json_str)